In [1]:
!pip install -q transformers==4.40.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 67.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 78.2 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.2 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.


In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import pandas as pd
from tqdm import tqdm
import math, os, torch

# ==========================
# 🔧 CONFIGURATION
# ==========================
start_idx, end_idx = 0, None  # <--- UPDATE: translate whole dataset
input_file = "/kaggle/input/gpcfiltered/GPC Filtered.xlsx"
output_file = f"translated__full.xlsx"

model_name = "Helsinki-NLP/opus-mt-en-ar"
batch_size = 32
save_every = 500  # Save progress every 500 rows

# ==========================
# 🚀 MODEL INITIALIZATION
# ==========================
print("🔄 Loading model and tokenizer...")
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print(f"✅ Model loaded on device: {device}")

# ==========================
# 📂 DATA LOADING
# ==========================
print(f"📖 Reading data from {input_file}...")
df = pd.read_excel(input_file)

# REMOVE the slicing — translate full dataset
df = df.reset_index(drop=True)

# Resume progress if exists
if os.path.exists(output_file):
    translated_df = pd.read_excel(output_file)
    print(f"🔁 Resuming from existing progress: {output_file}")
else:
    translated_df = pd.DataFrame()

# ==========================
# 🧠 TRANSLATION FUNCTION
# ==========================
def translate_batch(texts):
    # Skip empty or NaN entries
    texts = [t if isinstance(t, str) and t.strip() else "" for t in texts]
    prefixed_texts = [">>ara<< " + t for t in texts]

    inputs = tokenizer(prefixed_texts, return_tensors="pt", padding=True, truncation=True).to(device)
    translated = model.generate(**inputs, max_length=512)
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# ==========================
# ⚙ TRANSLATION LOOP
# ==========================
print("🌍 Starting translation process...\n")

for col in df.columns:
    print(f"🈶 Translating column: {col}")
    texts = df[col].astype(str).tolist()
    translated_texts = []

    for i in tqdm(range(0, len(texts), batch_size), desc=f"Translating {col}"):
        batch = texts[i:i + batch_size]
        translated_batch = translate_batch(batch)
        translated_texts.extend(translated_batch)

        # Save progress every save_every rows
        if i % (save_every * batch_size) == 0 and i > 0:
            partial_df = pd.concat([df, translated_df], axis=1)
            partial_df.to_excel(output_file, index=False)

    # Add translated column to translated_df
    translated_df[col + "_ar"] = translated_texts

# ==========================
# 💾 FINAL SAVE
# ==========================
final_df = pd.concat([df, translated_df], axis=1)
final_df.to_excel(output_file, index=False)

print(f"\n✅ Translation complete for!")
print(f"💾 Saved to: {output_file}")